<a href="https://colab.research.google.com/github/fgcortes/Projeto-da-Imersao-Alura/blob/main/Auxil_IA_de_Contrata%C3%A7%C3%B5es_Bot_para_Telegram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Instalação da biblioteca da API de bots para Telegram**

In [1]:
pip install pyTelegramBotAPI

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 1.8 MB/s eta 0:00:00


# **Instalação da biblioteca da API do Gemini**

In [2]:
pip install google-generativeai

# **Configuração do Gemini para uso como chatbot**

In [5]:
# Importação de módulos, classes e da biblioteca do Gemini

from pathlib import Path
import hashlib
import google.generativeai as genai

# Configuração da chave de API do Gemini usando o secret do Colab

from google.colab import userdata
api_key = userdata.get('API_GEMINI')
genai.configure(api_key=api_key)

# Configuração do modelo generativo

generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 10000,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

# Inserção de instruções para que o modelo aja e responda da forma esperada, ou seja, como um especialista em licitações e contratos

system_instruction = "Seu nome é Auxil.IA de contratações. Você é um especialista em licitações e contratos, especialmente na Lei nº 14.133/2021 e na jurisprudência do Tribunal de Contas da União. Sempre baseie suas respostas unicamente no conteúdo da referida lei e no arquivo PDF \"Licitacoes e Contratos - Orientacoes e Jurisprudencia do TCU - 5a Edicao.pdf\". A linguagem deve ser formal, mas de fácil entendimento. Não destaque os títulos e subtítulos de suas respostas com hashtag, use texto simples."

# Definição da versão do modelo a ser utilizada: Gemini 1.5 pro

model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              system_instruction=system_instruction,
                              safety_settings=safety_settings)

# Atribuição da fonte de dados (arquivos PDF) para basear as respostas

def extract_pdf_pages(pathname: str) -> list[str]:
  parts = [f"--- START OF PDF ${pathname} ---"]

  # Adição de lógica para ler os arquivos PDF e retornar a lista de páginas

  pages = []
  for index, page in enumerate(pages):
    parts.append(f"--- PAGE {index} ---")
    parts.append(page)
  return parts

# Exemplo do chat para definir o tipo de interação desejada

convo = model.start_chat(history=[
  {
    "role": "user",
    "parts": extract_pdf_pages("<path>/document0.pdf")
  },
  {
    "role": "user",
    "parts": extract_pdf_pages("<path>/document1.pdf")
  },
  {
    "role": "model",
    "parts": ["Como especialista em licitações e contratos, posso auxiliá-lo a navegar pela Lei nº 14.133/2021 e pelas orientações do TCU. Por favor, formule sua pergunta específica sobre o assunto, detalhando o seu contexto, para que eu possa fornecer uma resposta precisa e completa. \n\nLembre-se de que minhas respostas serão baseadas exclusivamente no conteúdo da Lei nº 14.133/2021 e no arquivo PDF \"Licitacoes e Contratos - Orientacoes e Jurisprudencia do TCU - 5a Edicao.pdf\"."]
  },
])

# Definição de saída do chat ao receber uma mensagem do usuário

convo.send_message("Olá")
print(convo.last.text)

Olá! 👋  Como posso ajudá-lo com seus questionamentos sobre licitações e contratos? 😊

Lembre-se: minhas respostas se baseiam na Lei nº 14.133/2021 e no documento "Licitações e Contratos - Orientações e Jurisprudência do TCU - 5ª Edição". 

Qual a sua dúvida? 😉 



# **Configuração do bot para Telegram, com interface com chatbot do Gemini**



In [ ]:
# Importação da biblioteca de bots do Telegram

import telebot

# Configuração da chave de API do bot do Telegram usando o secret do Colab

from google.colab import userdata
telegram_key = userdata.get('API_TELEGRAM')
bot = telebot.TeleBot(telegram_key)

# Configuração do manipulador de mensagens do bot e da função de resposta, para sempre responder as mensagens do usuário

@bot.message_handler(func=lambda m: True)

def echo_all(message):
  convo.send_message(message.text)
  response = (convo.last.text)
  bot.reply_to(message, response)

# Definição de parâmetro para permitir o funcionamento contínuo do bot

bot.infinity_polling()